In [12]:
using CUDA, Random
using GPUArrays: @allowscalar
CUDA.allowscalar(false)
include("utils.jl")
include("statistics.jl")

_var (generic function with 2 methods)

In [13]:
# data dimensions
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 6      # size of group 1
ny = 6      # size of group 2

# permutation test configuration
pooled = true
alpha = 0.05
deltas = repeat([0], B)
alternative = "two_sided"

# Generate data
Random.seed!(123)
x = randn(Float32, (B, S, nx))
y = randn(Float32, (B, S, ny))
px, py = partition(nx, ny)
wide = tconf.(eachslice(x, dims=1), eachslice(y, dims=1), alpha=0.0001)
wide = hcat(wide...)
narrow = tconf.(eachslice(x, dims=1), eachslice(y, dims=1), alpha=0.3)
narrow = hcat(narrow...)

# Move data to GPU
x = cu(x)
y = cu(y)
typeof(x)

CuArray{Float32, 3, CUDA.Mem.DeviceBuffer}

In [14]:
# get first batch for debugging purposes
x1, y1 = x[1,:,:], y[1,:,:]
x1.dims

(1800, 6)

In [15]:
function pval(x, y, pooled=false, alternative="two-sided", delta=0)
    """
    Parameters
    ----------
    x : Vector{Real}
        Data for group 1
    
    y : Vector{Real}
        Data for group 2
    
    pooled : Bool
        Assume equal/unequal variances for the two groups
    
    alternative : String
        Type of alternative hypothesis
    
    delta : Real
        Null hypothesis difference in means
    
    Returns
    -------
    Float64
        Proportion of pairs among all sample combinations which have
        a test statistic as or more extreme than the original pair (x, y)
    """
    
    x_shift = x .- delta              # shift group 1 under null hypothesis
    t_obs = t(x_shift, y, pooled)  # test statistic for observed data

    combined = vcat(x_shift, y)  # join original pair into single vector
    xs = combined[px]   # get all combinations of pairs from original pair
    ys = combined[py]
    ts = t(xs, ys, pooled)   # test statistic for all possible pairs of samples
    
    if alternative == "smaller"
        n_extreme = count(ts .<= t_obs)
    elseif alternative == "larger"
        n_extreme = count(ts .>= t_obs)
    else
        n_extreme = count(@. (ts <= -abs(t_obs)) | (ts >= abs(t_obs)))
    end

    return n_extreme / size(px, 1)  # proportion of pairs w/ extreme test statistic
    
end

pval (generic function with 4 methods)

In [16]:
function search(x, y, start, stop;
                pooled=false, alternative="two-sided", isLowerBound=true,
                margin=0.005, threshold=1.0, alpha=0.05)
    
    p_start = pval(x, y, pooled, alternative, start)
    p_end   = pval(x, y, pooled, alternative, stop)
    #println("p_start = ", p_start, ", p_end = ", p_end)
    
    # p-values corresponding to `start` and `stop` must be on opposite sides of `alpha`
    @assert (p_start - alpha) * (p_end - alpha) <= 0

    p = p_new = delta = nothing
    percent_change = (old, new) -> 100 * abs(new-old) / old
    
    i = 0
    while true
        i += 1
        delta = (start + stop) / 2
        p_new = pval(x, y, pooled, alternative, delta)

        if !isnothing(p) && percent_change(p, p_new) <= threshold
            break  # (1) percent change in p-value is below `threshold`
        end
        
        compare = (alpha - p_new) - isLowerBound * 2 * (alpha - p_new)
        if margin < compare
            stop = delta
        elseif margin < -compare
            start = delta
        else
            break  # (2) p-value is within `margin` of `alpha`
        end

        p = p_new
    end
    
    return delta
end

search (generic function with 1 method)

In [17]:
function permInterval(x, y, wide, narrow, delta_true; pooled=false, alpha=0.05, alternative="two_sided")
    """Returns true (false) if permutation test confidence interval does (not) include difference in
    population means.
    Parameters
    ----------
    x1 : Vector{Float64}
        Data for group 1
    x2 : Vector{Float64}
        Data for group 2
    partitions : Tuple{Matrix{Int64}, Matrix{Int64}}
        The i-th rows of x1[partitions[1]] and x2[partitions[2]] denote the i-th arrangement of
        the original (n1+n2) observations into two groups of size n1 and n2.
    delta_true : Float64
        Difference in population means
    pooled : Bool
        Assume pooled or unpooled variances
    alpha : Float64
        Significance level
    alternative : String
        Type of alternative hypothesis ("two-sided", "smaller", "larger")
    Returns
    -------
    Bool
        True (false) if permutation test confidence interval does (not) include difference in population means.
    """

    # use binary search to find approximate permutation test confidence interval
    lo = search(x, y, wide[1], narrow[1],
                pooled=pooled, alpha=alpha, alternative=alternative, isLowerBound=true)
    hi = search(x, y, narrow[2], wide[2],
                pooled=pooled, alpha=alpha, alternative=alternative, isLowerBound=false)
    # println("(", lo, ", ", hi, ")")
    return lo <= delta_true <= hi
end

permInterval (generic function with 1 method)

In [18]:
function coverage(xs, ys, wide, narrow, delta_true; pooled=false, alpha=0.05, alternative="two_sided")
    covered = permInterval.(eachrow(xs), eachrow(ys), wide, narrow, delta_true,
                            pooled=pooled, alpha=alpha, alternative=alternative)
    return count(covered) / S
end

coverage (generic function with 1 method)

In [19]:
results = coverage.(eachslice(x, dims=1),
                    eachslice(y, dims=1),
                    eachcol(wide),
                    eachcol(narrow),
                    deltas,
                    pooled=pooled,
                    alpha=alpha,
                    alternative=alternative)


outfile = "results2.txt"
open(outfile, "w") do file
    println(
        file,
        "=======================\n",
        "alternative : ", alternative, "\n",
        "pooled      : ", pooled, "\n",
        "alpha       : ", alpha, "\n",
        "======================="
    )
    for coverageProb in results
        println(file, coverageProb)
    end
end

LoadError: InterruptException: